In [ ]:
def guardar_resultados_test(resultados_test: dict, archivo_base=None):
    """
    Guarda los resultados de la evaluación en test en un archivo JSON.
    """


    def convertir_a_nativos(obj):
        if isinstance(obj, dict):
            return {convertir_a_nativos(k): convertir_a_nativos(v) for k, v in obj.items()}
        if isinstance(obj, list):
            return [convertir_a_nativos(x) for x in obj]
        if isinstance(obj, (np.integer, np.floating, np.bool_)):
            return obj.item()
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return obj

    # Convertir TODO a tipos nativos
    resultados_test = convertir_a_nativos(resultados_test)

    if archivo_base is None:
        archivo_base = STUDY_NAME

    resultados_dir = os.path.join(BUCKET_NAME, "resultados")
    os.makedirs(resultados_dir, exist_ok=True)

    archivo = os.path.join(resultados_dir, f"{archivo_base}_test_results.json")

    if os.path.exists(archivo):
        with open(archivo, 'r') as f:
            try:
                datos_existentes = json.load(f)
                if not isinstance(datos_existentes, list):
                    datos_existentes = []
            except json.JSONDecodeError:
                datos_existentes = []
    else:
        datos_existentes = []

    datos_existentes.append(resultados_test)

    with open(archivo, 'w') as f:
        json.dump(datos_existentes, f, indent=2)

    logger.info(
        f"Resultados guardados en {archivo} - Ganancia: {resultados_test['ganancia_test_promedio']:,.0f}"
    )
